# Import the needed libraries

In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import the Data

In [2]:
# Data imports
train_data = pd.read_csv(r'../project_data/train_data.csv')
test_data = pd.read_csv(r'../project_data/test_data.csv')

/var/folders/hr/bvcr3w9n2gxgqdhjgthjjnv80000gn/T/ipykernel_81756/1448665868.py:2: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(r'../project_data/train_data.csv')


### 1.1 Check data contents, provide descriptive statistics and check for incoherencies in the data.

In [17]:
# Check for column names in training data
print(train_data.columns)

# First 5 rows of the training data
train_data.head()

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [41]:
# Check for column names in test data
train_data.columns

# Find column names that don't exist in the test data but exist in the training data
missing_cols = [col for col in train_data.columns if col not in test_data.columns]

print("These will be the columns and the types of the columns that are not in testing data.\n"
    "We will try to predict these columns: \n" + str(missing_cols))


These will be the columns and the types of the columns that are not in testing data.
We will try to predict these columns: 
['Claim Injury Type', 'Agreement Reached', 'WCB Decision']


#### These will be the columns and description of the column from metadata that are not in testing data. These are the columns we are trying to predict: 
 
| Claim Injury Type | Agreement Reached | WCB Decision |
|------------------|-----------------------------------------------------------------|--------------|
|     **Main target variable**: Deliberation of the WCB relative to benefits awarded to the claim. **Numbering indicates severity**.       |     **Binary variable**: Yes if there is an agreement without the involvement of the WCB -> unknown at the start of a claim    |   **Multiclass variable**: Decision of the WCB relative to the claim: “Accident” means that claim refers to workplace accident, “Occupational Disease” means illness from the workplace. -> requires WCB deliberation so it is unknown at start of claim.    |



#### Let's take more comprehensive look into data contents and let's make observations